In [1]:
import cvxpy as cp
import numpy as np
import tqdm
import scipy
import math
from scipy.special import xlogy
import time

In [2]:
def channel_capacity(n, m, P, sum_x=1):
    '''
    Boyd and Vandenberghe, Convex Optimization, exercise 4.57 page 207
    Capacity of a communication channel.

    We consider a communication channel, with input X(t)∈{1,..,n} and
    output Y(t)∈{1,...,m}, for t=1,2,... .The relation between the
    input and output is given statistically:
    p_(i,j) = ℙ(Y(t)=i|X(t)=j), i=1,..,m  j=1,...,n

    The matrix P ∈ ℝ^(m*n) is called the channel transition matrix, and
    the channel is called a discrete memoryless channel. Assuming X has a
    probability distribution denoted x ∈ ℝ^n, i.e.,
    x_j = ℙ(X=j), j=1,...,n

    The mutual information between X and Y is given by
    ∑(∑(x_j p_(i,j)log_2(p_(i,j)/∑(x_k p_(i,k)))))
    Then channel capacity C is given by
    C = sup I(X;Y).
    With a variable change of y = Px this becomes
    I(X;Y)=  c^T x - ∑(y_i log_2 y_i)
    where c_j = ∑(p_(i,j)log_2(p_(i,j)))
    '''

    # n is the number of different input values
    # m is the number of different output values
    if n*m == 0:
        print('The range of both input and output values must be greater than zero')
        return 'failed', np.nan, np.nan

    # x is probability distribution of the input signal X(t)
    x = cp.Variable(shape=n)

    # y is the probability distribution of the output signal Y(t)
    # P is the channel transition matrix
    y = P@x

    # I is the mutual information between x and y
    c = np.sum(np.array((xlogy(P, P) / math.log(2))), axis=0)
    I = c@x + cp.sum(cp.entr(y) / math.log(2))

    # Channel capacity maximised by maximising the mutual information
    obj = cp.Maximize(I)
    constraints = [cp.sum(x) == sum_x,x >= 0]

    # Form and solve problem
    prob = cp.Problem(obj,constraints)
    prob.solve()
    if prob.status=='optimal':
        return prob.status, prob.value, x.value
    else:
        return prob.status, np.nan, np.nan

In [342]:
def calculate_base_D(px, i, j, k, S, N, Phi):
    if i == j:
        H_thres = [S[i-1], S[j]]
    H_thres = [S[i], S[j]]
    # H_thres = np.linspace(S[i], S[j], k+1)
    # H_thres[0] = S[0]
    # H_thres[-1] = S[-1]
    
    Ax = np.zeros((k, N))
    for m in range(N):
        for n in range(k):
            Ax[n, m] = Phi[m].cdf(H_thres[n+1]) - Phi[m].cdf(H_thres[n])
            
    # py = np.matmul(Ax, px)
    # Hy = -np.sum(np.array((xlogy(py, py) / math.log(2))), axis=0)
    
#     c = np.sum(np.array((xlogy(Ax, Ax) / math.log(2))), axis=0)
#     Hyx = -np.sum(px*c)
    
#     I = Hy - Hyx
    
    # Ay = (Ax*px)/np.sum(Ax*px, axis=1)
    # Ay = Ay.T

    # do this to handle divide by zero

    Ay = np.zeros((N, k))
    for m in range(N):
        for n in range(k):
            if np.round(np.sum(px*Ax[n,:]), 10) == 0:
                Ay[m, n] = 0
            else:
                Ay[m, n] = px[m]*Ax[n, m]/np.sum(px*Ax[n,:])

    py = np.matmul(Ax, px)
    c = np.sum(np.array((xlogy(Ay, Ay) / math.log(2))), axis=0)
    # Hxy = -np.sum(py*c)
    
    return -np.sum(py*c)

## init

In [299]:
X = np.array([-3, 0, 3])
Q = len(X)
N = 20
start = -8
end = 8
step = (end-start)/N
S = np.linspace(-8, 8, N+1)
M = 3

sigma = 0.5
Y = X + np.random.randn(Q)*sigma

Phi = [scipy.stats.norm(loc=X[i], scale=sigma) for i in range(Q)]

# thres = [-1.5, 1.5]
# thresholds = [S[0]] + thres +[S[-1]]

# A = np.zeros((M, Q))

# for j in range(Q):
#     for i in range(M):
#         A[i, j] = PhiY[j].cdf(thresholds[i+1]) - PhiY[j].cdf(thresholds[i])

In [304]:
px = [1/3, 1/3, 1/3]

In [343]:
D = np.zeros((N+1, M+1))
H = np.zeros((N+1, M+1))

In [348]:
for n in range(1, N+1):
    D[n, 1] = calculate_base_D(px, 0, n, 1, S, Q, Phi)
    H[n, 1] = 0
    
for m in range(2, M+1):
    for n in np.arange(m, N-M+m+1)[::-1]:
        tmp = []
        for t in range(m-1, n):
            # print(t, m-1,D[t, m-1])
            tmp.append(D[t, m-1] + calculate_base_D(px, t+1, n, 1, S, Q, Phi))
        # print(tmp)
        h = np.argmin(tmp)
        H[n, m] = h
        D[n, m] = D[h, m-1] + calculate_base_D(px, h+1, n, 1, S, Q, Phi)
        
H = H.astype(int)

In [354]:
D[1:,1:]

array([[-0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.89617602e-28,  4.89617085e-28,  4.89617602e-28],
       [ 7.79001386e-21,  7.78989305e-21,  7.79001386e-21],
       [ 9.64179885e-15,  9.64001596e-15,  9.64179624e-15],
       [ 8.67239237e-10,  8.64680760e-10,  8.67221979e-10],
       [ 5.69395689e-06,  5.50526080e-06,  5.68402060e-06],
       [ 2.73632379e-03,  2.01420472e-03,  2.59633368e-03],
       [ 1.03592815e-01,  5.00629047e-03,  5.13660088e-02],
       [ 4.59147927e-01,  7.71945550e-03,  1.42559000e-01],
       [ 6.48064886e-01,  8.53525191e-03,  1.75142096e-01],
       [ 6.75853652e-01,  1.71366223e-02,  1.27200520e-02],
       [ 8.81572197e-01,  1.86723440e-01,  1.47565289e-01],
       [ 1.38027171e+00,  5.23628260e-01,  3.85104640e-01],
       [ 1.57285932e+00,  6.43460575e-01,  4.59676211e-01],
       [ 1.58487844e+00,  6.50750296e-01,  4.64123192e-01],
       [ 1.58496245e+00,  6.50801178e-01

In [350]:
H

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 2, 1],
       [0, 0, 3, 2],
       [0, 0, 4, 3],
       [0, 0, 5, 4],
       [0, 0, 6, 5],
       [0, 0, 7, 6],
       [0, 0, 7, 6],
       [0, 0, 7, 6],
       [0, 0, 7, 9],
       [0, 0, 7, 9],
       [0, 0, 8, 9],
       [0, 0, 8, 9],
       [0, 0, 8, 9],
       [0, 0, 8, 9],
       [0, 0, 8, 9],
       [0, 0, 9, 9],
       [0, 0, 0, 9]])

In [351]:
h_opt = []
h_prev = N
h_opt.append(h_prev)

for m in np.arange(1, M+1)[::-1]:
    h_prev = H[h_prev, m]
    h_opt.append(h_prev)

In [352]:
thres = []
for h in h_opt[::-1]:
    thres.append(S[h])
print(thres)

[-8.0, -2.3999999999999995, -0.7999999999999998, 8.0]


In [335]:
np.arange(M)[::-1]

array([2, 1, 0])

In [71]:
S[19]

-0.39999999999999947

In [72]:
S[13]

-2.8

In [20]:
H

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 2.,  1.,  0.],
       [ 3.,  2.,  1.],
       [ 4.,  3.,  2.],
       [ 5.,  4.,  3.],
       [ 6.,  5.,  4.],
       [ 7.,  6.,  5.],
       [ 8.,  7.,  6.],
       [ 9.,  8.,  7.],
       [10.,  8.,  8.],
       [11., 10.,  9.],
       [12., 11., 10.],
       [13., 12., 11.],
       [14., 12., 12.],
       [15., 12., 12.],
       [16., 12., 12.],
       [ 0., 12., 12.],
       [ 0.,  0., 12.],
       [ 0.,  0.,  0.]])

In [15]:
N

3